In [1]:
from langchain_ollama import ChatOllama

model = ChatOllama(model = "gemma:2b")

model

ChatOllama(model='gemma:2b')

In [16]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import PydanticOutputParser

class Information(BaseModel):
    about: str = Field(description="One-line intro of the topic")
    advantages: List[str] = Field(description="Exactly 3 advantages")
    facts: List[str] = Field(description="3 factual points")

parser = PydanticOutputParser(pydantic_object=Information)


In [18]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate(
    template="""
You MUST return ONLY a JSON object with actual values.
DO NOT return schema, properties, types, or explanations.

{format_instructions}

Topic: {topic}
""",
    input_variables=["topic"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)

template

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"about": {"description": "One-line intro of the topic", "title": "About", "type": "string"}, "advantages": {"description": "Exactly 3 advantages", "items": {"type": "string"}, "title": "Advantages", "type": "array"}, "facts": {"description": "3 factual points", "items": {"type": "string"}, "title": "Facts", "type": "array"}}, "required": ["about", "advantages", "facts"]}\n```'}, template='\nYou MUST return ONLY a JSON obj

In [19]:
topic = input("Enter the topic to search: ")

prompt = template.invoke({"topic": topic})

prompt


Enter the topic to search:  Python


StringPromptValue(text='\nYou MUST return ONLY a JSON object with actual values.\nDO NOT return schema, properties, types, or explanations.\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"about": {"description": "One-line intro of the topic", "title": "About", "type": "string"}, "advantages": {"description": "Exactly 3 advantages", "items": {"type": "string"}, "title": "Advantages", "type": "array"}, "facts": {"description": "3 factual points", "items": {"type": "string"}, "title": "Facts", "type": "array"}}, "required": ["about", "advantages", "facts"]}\n```\n\nT

In [20]:
response = model.invoke(prompt)
print(response)

content='{"properties": {"about": {"description": "One-line intro of the topic", "title": "About", "type": "string"}, "advantages": {"description": "Exactly 3 advantages", "items": {"type": "string"}, "title": "Advantages", "type": "array"}, "facts": {"description": "3 factual points", "items": {"type": "string"}, "title": "Facts", "type": "array"}, "required": ["about", "advantages", "facts"]}' additional_kwargs={} response_metadata={'model': 'gemma:2b', 'created_at': '2025-12-13T12:45:11.2134384Z', 'done': True, 'done_reason': 'stop', 'total_duration': 35944062800, 'load_duration': 222052000, 'prompt_eval_count': 282, 'prompt_eval_duration': 19008441400, 'eval_count': 108, 'eval_duration': 15980043600, 'logprobs': None, 'model_name': 'gemma:2b', 'model_provider': 'ollama'} id='lc_run--019b17bd-d443-7852-be2e-f29ea29b732d-0' usage_metadata={'input_tokens': 282, 'output_tokens': 108, 'total_tokens': 390}


In [21]:
chain = template | model | parser
response = chain.invoke({"topic": "Python"})
print(response)

OutputParserException: Failed to parse Information from completion {"properties": {"about": {"description": "One-line intro of the topic", "title": "About", "type": "string"}, "advantages": {"description": "Exactly 3 advantages", "items": {"type": "string"}, "title": "Advantages", "type": "array"}, "facts": {"description": "3 factual points", "items": {"type": "string"}, "title": "Facts", "type": "array"}, "required": ["about", "advantages", "facts"]}}. Got: 3 validation errors for Information
about
  Field required [type=missing, input_value={'properties': {'about': ...'advantages', 'facts']}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
advantages
  Field required [type=missing, input_value={'properties': {'about': ...'advantages', 'facts']}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
facts
  Field required [type=missing, input_value={'properties': {'about': ...'advantages', 'facts']

Gemma (Ollama) does NOT reliably follow PydanticOutputParser instructions.
It keeps returning the JSON schema instead of actual JSON values.
This is a model limitation, not your code.

DO NOT use PydanticOutputParser with Gemma

Use manual JSON + JsonOutputParser instead.

so,

Small Ollama models → JSONOutputParser

Large OpenAI/Claude → PydanticOutputParser